In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import polars as pl
from datetime import datetime, timedelta
import gc
from temporal_fusion_transformer.src.experiments.favorita import read_temporal, downcast_dataframe

start_date = datetime(2016, 1, 1)
end_date = datetime(2016, 6, 1)
validation_boundary = datetime(2016, 4, 1)
total_time_steps: int = 120
num_encoder_steps: int = 90
forecast_horizon = total_time_steps - num_encoder_steps
test_boundary = validation_boundary + timedelta(hours=forecast_horizon)
forecast_horizon = total_time_steps - num_encoder_steps
data_dir = "../data/favorita"

In [2]:
gc.collect()

0

In [3]:
df = pl.read_parquet(
    "/Users/artemsereda/Documents/IdeaProjects/temporal_fusion_transformer/data/favorita/df.parquet"
)

In [4]:
df

date,store_nbr,item_nbr,log_sales,onpromotion,traj_id,open,oil_price,city,state,type,cluster,family,class,perishable,transactions,national_hol,regional_hol,local_hol,month,day_of_month,day_of_week
date,u8,u32,f32,u8,str,i8,f32,str,str,str,u8,str,u16,u8,u16,str,str,str,u32,u32,u32
2016-01-02,16,1001305,1.098612,0,"""16_1001305""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1016,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1003679,0.693147,0,"""16_1003679""",1,49.360001,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1028,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1004550,1.386294,0,"""16_1004550""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""LIQUOR,WINE,BE…",1318,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1004551,1.098612,0,"""16_1004551""",1,49.360001,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""LIQUOR,WINE,BE…",1318,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1009512,0.0,0,"""16_1009512""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""AUTOMOTIVE""",6810,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1012473,3.7612,0,"""16_1012473""",1,47.669998,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""CLEANING""",3008,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1012787,0.0,0,"""16_1012787""",1,49.099998,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1012,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1012788,0.693147,1,"""16_1012788""",1,49.360001,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1012,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1014865,0.693147,0,"""16_1014865""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""DAIRY""",2108,1,373,"""""","""""","""""",1,2,6


In [8]:
training_df: pl.DataFrame = df.filter(pl.col("date").over("traj_id").lt(validation_boundary))

In [9]:
training_df

date,store_nbr,item_nbr,log_sales,onpromotion,traj_id,open,oil_price,city,state,type,cluster,family,class,perishable,transactions,national_hol,regional_hol,local_hol,month,day_of_month,day_of_week
date,u8,u32,f32,u8,str,i8,f32,str,str,str,u8,str,u16,u8,u16,str,str,str,u32,u32,u32
2016-01-02,16,1001305,1.098612,0,"""16_1001305""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1016,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1003679,0.693147,0,"""16_1003679""",1,49.360001,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1028,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1004550,1.386294,0,"""16_1004550""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""LIQUOR,WINE,BE…",1318,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1004551,1.098612,0,"""16_1004551""",1,49.360001,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""LIQUOR,WINE,BE…",1318,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1009512,0.0,0,"""16_1009512""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""AUTOMOTIVE""",6810,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1012473,3.7612,0,"""16_1012473""",1,47.669998,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""CLEANING""",3008,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1012787,0.0,0,"""16_1012787""",1,49.099998,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1012,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1012788,0.693147,1,"""16_1012788""",1,49.360001,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""GROCERY I""",1012,0,373,"""""","""""","""""",1,2,6
2016-01-02,16,1014865,0.693147,0,"""16_1014865""",1,49.07,"""Santo Domingo""","""Santo Domingo …","""C""",3,"""DAIRY""",2108,1,373,"""""","""""","""""",1,2,6


In [10]:
validation_df = df.filter(pl.col("date").over("traj_id").ge(validation_boundary)).filter(
    pl.col("date").over("traj_id").lt(test_boundary)
)
test_df = df.filter(pl.col("date").over("traj_id").ge(test_boundary))

In [11]:
identifiers = training_df["traj_id"].unique().to_list()
ids = set(identifiers)

In [12]:
ids

{'46_1913230',
 '31_319272',
 '44_964752',
 '33_1386506',
 '34_1929797',
 '38_1963277',
 '19_1416362',
 '2_119023',
 '23_372924',
 '49_1402017',
 '43_1143691',
 '50_730258',
 '34_1447815',
 '31_1501570',
 '12_1463788',
 '7_849135',
 '2_956013',
 '29_368424',
 '48_1372585',
 '35_1391408',
 '34_2010743',
 '4_1309672',
 '24_1940026',
 '31_1166036',
 '28_1386894',
 '4_724403',
 '49_1311104',
 '24_323013',
 '48_843610',
 '50_215332',
 '26_562568',
 '20_1370564',
 '25_1124165',
 '2_1464045',
 '39_1012473',
 '8_838842',
 '34_2010989',
 '17_510052',
 '11_1239783',
 '29_1962621',
 '28_1473410',
 '26_121964',
 '38_323217',
 '32_361502',
 '51_587156',
 '26_1914786',
 '48_1920284',
 '27_1370547',
 '3_407769',
 '5_1579081',
 '49_1047680',
 '21_1386506',
 '49_1971730',
 '36_1457181',
 '14_953404',
 '18_1372511',
 '53_1920071',
 '13_820500',
 '39_841842',
 '24_1316369',
 '36_940591',
 '38_634015',
 '12_514172',
 '2_2026945',
 '18_1920072',
 '33_570917',
 '41_890825',
 '2_1152348',
 '5_1331068',
 '1_1

In [15]:
def filter_ids(frame: pl.DataFrame) -> pl.DataFrame:
    return frame.filter(pl.col("traj_id").is_in(ids))


validation_df = filter_ids(validation_df)
test_df = filter_ids(test_df)

In [16]:
validation_df

date,store_nbr,item_nbr,log_sales,onpromotion,traj_id,open,oil_price,city,state,type,cluster,family,class,perishable,transactions,national_hol,regional_hol,local_hol,month,day_of_month,day_of_week
date,u8,u32,f32,u8,str,i8,f32,str,str,str,u8,str,u16,u8,u16,str,str,str,u32,u32,u32
2016-04-01,10,1001305,1.386294,0,"""10_1001305""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""GROCERY I""",1016,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1003679,1.94591,0,"""10_1003679""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""GROCERY I""",1028,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1004550,0.693147,0,"""10_1004550""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LIQUOR,WINE,BE…",1318,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1004551,0.693147,0,"""10_1004551""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LIQUOR,WINE,BE…",1318,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1005458,1.386294,0,"""10_1005458""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1005461,0.0,0,"""10_1005461""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1005463,0.693147,0,"""10_1005463""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1005465,0.0,0,"""10_1005465""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,842,"""""","""""","""""",4,1,5
2016-04-01,10,1009512,0.0,0,"""10_1009512""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""AUTOMOTIVE""",6810,0,842,"""""","""""","""""",4,1,5


In [17]:
test_df

date,store_nbr,item_nbr,log_sales,onpromotion,traj_id,open,oil_price,city,state,type,cluster,family,class,perishable,transactions,national_hol,regional_hol,local_hol,month,day_of_month,day_of_week
date,u8,u32,f32,u8,str,i8,f32,str,str,str,u8,str,u16,u8,u16,str,str,str,u32,u32,u32
2016-04-03,10,1001305,2.079442,1,"""10_1001305""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""GROCERY I""",1016,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1003679,1.386294,0,"""10_1003679""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""GROCERY I""",1028,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1004550,0.0,0,"""10_1004550""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LIQUOR,WINE,BE…",1318,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1004551,1.098612,0,"""10_1004551""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LIQUOR,WINE,BE…",1318,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1005458,1.386294,0,"""10_1005458""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1005461,0.0,0,"""10_1005461""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1005463,0.0,0,"""10_1005463""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1005465,0.0,0,"""10_1005465""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""LINGERIE""",7002,0,1105,"""""","""""","""""",4,3,7
2016-04-03,10,1009512,0.0,0,"""10_1009512""",1,35.360001,"""Quito""","""Pichincha""","""C""",15,"""AUTOMOTIVE""",6810,0,1105,"""""","""""","""""",4,3,7
